In [1]:
from tensorflow.keras.models import load_model

In [2]:
path = r'E:\project_24\Major_Project_forgery\code\models\model_casia_run1.h5'
model = load_model(path)
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 124, 124, 32)      2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 120, 120, 32)      25632     
                                                                 
 max_pooling2d (MaxPooling2  (None, 60, 60, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 60, 60, 32)        0         
                                                                 
 flatten (Flatten)           (None, 115200)            0         
                                                                 
 dense (Dense)               (None, 256)               29491456  
                                                      

In [4]:
from common.helper import *
from database import *

In [8]:
db = open_db()
class_names=['authentic', 'tampered']
img = db.query(File).get(1)
path = img.path
if os.path.exists(path):
    image = prepare_image(path)
    image = image.reshape(-1, 128, 128, 3)
    model = load_my_model('models\model_casia_run1.h5')
    print('model loaded')
    y_pred = model.predict(image)
    y_pred_class = np.argmax(y_pred, axis=1)[0]
    cls = class_names[y_pred_class]
    conf  = np.amax(y_pred)*100
    if y_pred_class == 0:
        try:
            os.makedirs('static/ela', exist_ok=True)
            ela_path = f'static/ela/{img.id}.png'
            convert_to_ela_image(path, 91).save(ela_path)
            coords = find_manipulated_region(ela_path)
            modify_boundary = make_pixels_white(ela_path, coords)
            os.makedirs('static/modified', exist_ok=True)
            modified_path = f'static/modified/{img.id}.png'
            modify_boundary.save(modified_path)
        except Exception as e:
            print("😠",e)
    print(f'{cls} with confidence {conf:.2f}%')


2024-04-13 16:24:18,390 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-13 16:24:18,393 INFO sqlalchemy.engine.Engine SELECT files.id AS files_id, files.path AS files_path, files.user_id AS files_user_id, files.created_at AS files_created_at 
FROM files 
WHERE files.id = ?
2024-04-13 16:24:18,394 INFO sqlalchemy.engine.Engine [generated in 0.00097s] (1,)


C:\Users\ZAID\AppData\Local\Temp\ipykernel_9224\4254781881.py:3: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  img = db.query(File).get(1)


model loaded
1/1 [==============================] - 0s 64ms/step
😠 ufunc 'greater' did not contain a loop with signature matching types (<class 'numpy.dtypes.StrDType'>, <class 'numpy.dtypes.Int32DType'>) -> None
authentic with confidence 64.25%
